In [29]:
import numpy as np
m = np.array([[7, 12, 1, 14], [2, 13, 8, 11], [16, 3, 10, 5], [9, 6, 15, 4]])

# Zadanie 1. Suma elementów macierzy

In [30]:
def matrix_sum(A):
    sum_of_all = sum(sum(A))
    return sum_of_all


In [31]:
m.sum()

136

In [32]:
matrix_sum(m)

136

# Zadanie 2. Suma diagonali

In [10]:
def sum_diag(A):
    '''
    Sums all elements over the matrix diagonal.
    Inputs: numpy matrix of any shape
    Outputs: sum of its diagonal elements
    '''
    height = A.shape[0]
    res = 0
    for h in range(height):
        res += A[h, h]
    return res

sum_diag(m)

34

# Zadanie 3. Suma antydiagonali

In [11]:
def sum_antidiag(A):
    '''
    Sums all elements over the matrix antidiagonal.
    Inputs: numpy matrix of any shape
    Outputs: sum of its diagonal elements
    '''
    height = A.shape[0]
    res = 0
    for h in range(height):
        res += A[h, height - 1 - h]
    return res

sum_antidiag(m)

34

# Zadanie 4. Wyznacznik macierzy 

In [12]:
def determine(A):
    '''
    Calculates the determinant of any square matrix.
    Input: A square matrix of type numpy.ndarray
    Output: A numeric value
    '''
    # Check if input is a numpy array
    if(not isinstance(m, np.ndarray)):
        raise ValueError('Input must be of type numpy.ndarray')  
    # Check if matrix is square
    if(A.shape[0] != A.shape[1] ):
        raise ValueError('Input must be a square matrix')   
    # Check if matrix has 2 dimensions
    if(len(A.shape) != 2):
        raise ValueError('The matrix must be two-dimensional') 
    
    height  = A.shape[0]
    # If the matrix is 2x2 retrurn its determiner
    if(height == 2):
        minuend = A[0, 0] * A[1, 1] 
        subtrahend = A[0, 1] * A[1, 0]
        diff = minuend - subtrahend
        return diff
    # Otherwise reduce it to sum of products
    else:        
        # Remove the first row
        A_short = np.delete(A, 0, 0)
        #Initialise the sum of results
        res = 0
        for h in range(height):
            # Remove the h'th column
            to_det = np.delete(A_short, h, 1)
            # Take the determiner of the matrix
            det = determine(to_det)
            prod = det * A[0, h]
            # Add products of odd columns
            if(h % 2 == 0):
                res += prod
            # Subtract products of even columns
            else:
                res -= prod
        return res

print(np.array([[1,2,4], [2,-1,3], [4,0,1]]))
determine(np.array([[1,2,4], [2,-1,3], [4,0,1]]))

[[ 1  2  4]
 [ 2 -1  3]
 [ 4  0  1]]


35

# Zadanie 5. Odwrotność Macierzy

#### Kalkulacje w etapach

In [34]:
# https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html
def matrix_of_minors(A):
    # Loop over each element in the matrix
    length = A.shape[0]
    # Initialise matrix of results
    M_of_minors = np.zeros(A.shape)
    for i in range(length):
        A_short = np.delete(A, i, 0)
        for j in range(length):
            A_small = np.delete(A_short, j, 1)
            #print(A_small)
            #print("A_small is \n",A_small)
            M_of_minors[i, j] = determine(A_small)
            #print(M_of_minors[i, j])
    return M_of_minors

print(np.array([[3, 0, 2],[2, 0, -2], [0, 1, 1]]))
matrix_of_minors(np.array([[3, 0, 2],[2, 0, -2], [0, 1, 1]]))

[[ 3  0  2]
 [ 2  0 -2]
 [ 0  1  1]]


array([[  2.,   2.,   2.],
       [ -2.,   3.,   3.],
       [  0., -10.,   0.]])

In [14]:
def minors_to_cof(A):
    length = A.shape[0]
    sign = -1
    for i in range(length):
        sign *= -1
        for j in range(length):
            A[i, j] *= sign 
            sign *= -1
    return A

In [15]:
def transpose(A):
    length = A.shape[0]
    B = A
    for i in range(length):
        for j in range(length):
            B[j, i] = A[i, j] 
    return B

In [16]:
def invert(A):
    mm = matrix_of_minors(A)
    mc = minors_to_cof(mm)
    tmc = transpose(mm)
    inv_A = tmc/determine(A)
    return inv_A

#### Kalkulacje zintegrowane

In [117]:
# https://www.mathsisfun.com/algebra/matrix-inverse-minors-cofactors-adjugate.html
def inv_matrix(A):
    '''
    Calculates the inverse of an invertible square matrix.
    Input: A square matrix of type numpy.ndarray
    Output: A numpy ndarray, which is an inverse of the matrix
    '''
    # Check if input is a numpy array
    if(not isinstance(m, np.ndarray)):
        raise ValueError('Input must be of type numpy.ndarray')  
    # Check if matrix is square
    if(A.shape[0] != A.shape[1] ):
        raise ValueError('Input must be a square matrix')   
    # Check if matrix has 2 dimensions
    if(len(A.shape) != 2):
        raise ValueError('The matrix must be two-dimensional')
    #!!! We should also check if any two columns or any two rows are the same !!!
    
    # Loop over each element in the matrix
    length = A.shape[0]
    # Initialise matrix of results
    M_adj = np.zeros(A.shape)
    # Initialise row sign change
    row_sign = -1
    # Initialise vector of determinants
    top3dets = np.zeros(length)
    
    # Row loop
    for i in range(length):
        # Remove the value's row
        A_short = np.delete(A, i, 0)
        # Initialise row sign for cofactors
        row_sign *= -1
        # Make the first value sign the same as row_sign
        sign = row_sign
        
        # Column/value loop
        for j in range(length):
            # Remove value's column
            A_small = np.delete(A_short, j, 1)
            
            # The values j and i are switched instead for step 3 (adjugate) 
            M_adj[j, i] = determine(A_small) * sign
            # Save cofactors from the top row to calc det for entire matrix
            if(i == 0):
                top3dets[j] = M_adj[j, i]
            sign *= -1
            
    # Calculate the entire matrix's determinant
    big_det = sum(np.array(top3dets)* A[0, :])
    A_inv = M_adj/big_det    
    return A_inv



B = np.array([[3, 0, 2],[2, 0, -2], [0, 1, 1]])
inv_matrix(B)

array([[ 0.2,  0.2,  0. ],
       [-0.2,  0.3,  1. ],
       [ 0.2, -0.3,  0. ]])